# Import packages

In [1]:
# -----------------------------------------------------------------------------
# region import packages

# management
import glob
import pickle
import warnings
warnings.filterwarnings('ignore')
import os
import sys  # print(sys.path)
sys.path.append('/work/ollie/qigao001')
os.chdir('/home/users/qino')

# data analysis
import numpy as np
import xarray as xr
import dask
dask.config.set({"array.slicing.split_large_chunks": True})
# from dask.diagnostics import ProgressBar
# pbar = ProgressBar()
# pbar.register()
from scipy import stats
import xesmf as xe
import pandas as pd
from metpy.interpolate import cross_section
from statsmodels.stats import multitest
import pycircstat as circ
from scipy.stats import circstd
import cmip6_preprocessing.preprocessing as cpp

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib import cm
import cartopy.crs as ccrs
plt.rcParams['pcolor.shading'] = 'auto'
mpl.rcParams['figure.dpi'] = 600
mpl.rc('font', family='Times New Roman', size=10)
mpl.rcParams['axes.linewidth'] = 0.2
plt.rcParams.update({"mathtext.fontset": "stix"})
import matplotlib.animation as animation
import seaborn as sns
from matplotlib.ticker import AutoMinorLocator

# self defined
from a_basic_analysis.b_module.mapplot import (
    globe_plot,
    hemisphere_plot,
    quick_var_plot,
    mesh2plot,
    framework_plot1,
    remove_trailing_zero,
    remove_trailing_zero_pos,
)

from a_basic_analysis.b_module.basic_calculations import (
    mon_sea_ann,
    find_ilat_ilon,
    regrid,
    time_weighted_mean,
)

from a_basic_analysis.b_module.namelist import (
    month,
    month_num,
    month_dec,
    month_dec_num,
    seasons,
    seasons_last_num,
    hours,
    months,
    month_days,
    zerok,
    panel_labels,
    seconds_per_d,
)

from a_basic_analysis.b_module.source_properties import (
    source_properties,
    calc_lon_diff,
)

from a_basic_analysis.b_module.statistics import (
    fdr_control_bh,
    check_normality_3d,
    check_equal_variance_3d,
    ttest_fdr_control,
)

from a_basic_analysis.b_module.component_plot import (
    cplot_ice_cores,
    plt_mesh_pars,
)

# endregion
# -----------------------------------------------------------------------------


# import data

In [2]:
with open('scratch/cmip6/lig/sst/lig_pi_sst_regrid_alltime.pkl', 'rb') as f:
    lig_pi_sst_regrid_alltime = pickle.load(f)

with open('scratch/cmip6/lig/sst/sst_regrid_alltime_ens_stats.pkl', 'rb') as f:
    sst_regrid_alltime_ens_stats = pickle.load(f)

cdo_area1deg = xr.open_dataset('scratch/others/one_degree_grids_cdo_area.nc')
lat = lig_pi_sst_regrid_alltime['ACCESS-ESM1-5']['am'].lat.values
mask_so = (lat < -40)

# Model ensemble conditions

## SO annual SST

In [23]:
mean_value = np.ma.average(
    np.ma.MaskedArray(
        sst_regrid_alltime_ens_stats['lig_pi']['am']['mean'][0].values[mask_so],
        mask = np.isnan(sst_regrid_alltime_ens_stats['lig_pi']['am']['mean'][0].values[mask_so])),
    weights=cdo_area1deg.cell_area.values[mask_so])

std_values = np.ma.std(
    np.ma.MaskedArray(
        sst_regrid_alltime_ens_stats['lig_pi']['am']['mean'][0].values[mask_so],
        mask = np.isnan(sst_regrid_alltime_ens_stats['lig_pi']['am']['mean'][0].values[mask_so])),
)

print(str(np.round(mean_value, 1)) + ' ± ' + str(np.round(std_values, 1)))

0.1 ± 0.3


## SO summer SST

In [29]:
mean_value = np.ma.average(
    np.ma.MaskedArray(
        sst_regrid_alltime_ens_stats['lig_pi']['sm']['mean'][0].values[mask_so],
        mask = np.isnan(sst_regrid_alltime_ens_stats['lig_pi']['sm']['mean'][0].values[mask_so])),
    weights=cdo_area1deg.cell_area.values[mask_so])

std_values = np.ma.std(
    np.ma.MaskedArray(
        sst_regrid_alltime_ens_stats['lig_pi']['sm']['mean'][0].values[mask_so],
        mask = np.isnan(sst_regrid_alltime_ens_stats['lig_pi']['sm']['mean'][0].values[mask_so])),
)

print(str(np.round(mean_value, 1)) + ' ± ' + str(np.round(std_values, 1)))

-0.1 ± 0.4


## SO Sep SIC

In [ ]:
with open('scratch/cmip6/lig/sic/sic_regrid_alltime_ens_stats.pkl', 'rb') as f:
    sic_regrid_alltime_ens_stats = pickle.load(f)

In [36]:
mean_value = np.ma.average(
    np.ma.MaskedArray(
        sic_regrid_alltime_ens_stats['lig_pi']['mm']['mean'][8].values[mask_so],
        mask = np.isnan(sic_regrid_alltime_ens_stats['lig_pi']['mm']['mean'][8].values[mask_so])),
    weights=cdo_area1deg.cell_area.values[mask_so])

std_values = np.ma.std(
    np.ma.MaskedArray(
        sic_regrid_alltime_ens_stats['lig_pi']['mm']['mean'][8].values[mask_so],
        mask = np.isnan(sic_regrid_alltime_ens_stats['lig_pi']['mm']['mean'][8].values[mask_so])),
)

print(str(np.round(mean_value, 1)) + ' ± ' + str(np.round(std_values, 1)))

-2.5 ± 5.0


## AIS annual SAT

In [4]:
with open('scratch/others/land_sea_masks/cdo_1deg_ais_mask.pkl', 'rb') as f:
    cdo_1deg_ais_mask = pickle.load(f)

with open('scratch/cmip6/lig/tas/tas_regrid_alltime_ens_stats.pkl', 'rb') as f:
    tas_regrid_alltime_ens_stats = pickle.load(f)

In [14]:
data = tas_regrid_alltime_ens_stats['lig_pi']['am']['mean'][0].values
mask_ais = cdo_1deg_ais_mask['mask']['AIS']

mean_value = np.ma.average(
    np.ma.MaskedArray(
        data[mask_ais],
        mask = np.isnan(data[mask_ais])),
    weights=cdo_area1deg.cell_area.values[mask_ais])

std_values = np.ma.std(
    np.ma.MaskedArray(
        data[mask_ais],
        mask = np.isnan(data[mask_ais])),
)

print(str(np.round(mean_value, 1)) + ' ± ' + str(np.round(std_values, 1)))

0.5 ± 0.1
